In [1]:
!pip install bitsandbytes
!pip install -q datasets loralib sentencepiece
!pip install -q git+https://github.com/huggingface/transformers.git
!pip install -q git+https://github.com/huggingface/peft.git@64f63a7df2a02cfd144592d9aa9c871b59258c55
!pip install gradio
!pip install scipy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.3/104.3 MB 28.3 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 21.3 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 286.2/286.2 kB 62.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.5/50.5 kB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.8/57.8 kB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 35.8 MB/s eta 0:00:0000:010:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 813.6/813.6 kB 113.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 60.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 271.6/271.6 kB 28.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.3/75.3 kB 27.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 103.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.5/84.5 kB 34.1 MB/s eta 0:00:00


In [3]:
from peft import PeftModel
from transformers import LlamaTokenizer, LlamaForCausalLM, GenerationConfig
import torch

tokenizer = LlamaTokenizer.from_pretrained("chavinlo/alpaca-native")
model = LlamaForCausalLM.from_pretrained(
    "chavinlo/alpaca-native",
    load_in_8bit=True,
    torch_dtype=torch.float16,
    device_map="auto",
)
model = PeftModel.from_pretrained(model, "./llama-doctor", device_map={'': 0})

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'LLaMATokenizer'. 
The class this function is called from is 'LlamaTokenizer'.


Loading checkpoint shards:   0%|          | 0/33 [00:00<?, ?it/s]

In [4]:
def make_prompt(instruction):
        return f"""Below is an instruction that describes a task. Write a response that appropriately completes the request. 

### Instruction:
{instruction}

### Response:"""

In [20]:
def evaluate(q):
    input_ids = tokenizer(make_prompt(q), return_tensors="pt")["input_ids"].to('cuda')
    with torch.no_grad():
        gen_tokens = model.generate(
            input_ids=input_ids,
            max_new_tokens = 256,
            do_sample=True,
            temperature=0.5,
            top_k=20,
            repetition_penalty=1.2,
            #eos_token_id=0, # for open-end generation.
            #pad_token_id=tokenizer.eos_token_id,
        )
    origin_output = tokenizer.batch_decode(gen_tokens)[0]
    output = origin_output.split("### Response:")[1].strip()
    return output

In [ ]:
#@title GUI
import gradio as gr
examples = [
    ["Doctor, I have been experiencing vomiting, nausea, and abdominal pain. I recently found out that I have pyloric stenosis. What medications should I take for it?"],
    ["Hi doctor, I have been losing hair rapidly for the past few months. I think I might have Alopecia. What should I do?"],
    ["Doctor, I’ve been experiencing sudden jerks and twitches in my muscles. It’s becoming very difficult to manage them on a daily basis."],
    ["I think I have carbon monoxide poisoning. I feel dizzy and nauseous"],
    ["I have been experiencing memory problems and confusion lately. I think I might have Wernicke Korsakoff syndrome"],
]

title = "Chat Doctor"


def inference(instruction):
  return evaluate(instruction)

io = gr.Interface(
  inference,
  inputs=[gr.Textbox(lines=5)],
  outputs=[
    gr.Textbox(lines=5, label="Chat Doctor")
  ],
  title=title,
  examples=examples
)
io.launch(share=True)